In [1]:
import requests 
import numpy as np

### Getting the data from the API:

In [13]:
drivers=[]

url = 'http://ergast.com/api/f1/2018/drivers.json'
r = requests.get(url)

results = r.json()

for driver in results['MRData']['DriverTable']['Drivers']:
    drivers.append(str(driver['driverId']))

data = {}

for driver in drivers:

    url = 'http://ergast.com/api/f1/2018/drivers/'+driver+'/results.json'
    r = requests.get(url)

    results = r.json()

    races = []
    raceindex = []
    points = []
    team = []

    for race in results['MRData']['RaceTable']['Races']:
        races.append(race['raceName'])
        raceindex.append(int(race['round']))
        points.append(int(race['Results'][0]['points']))
        team.append(race['Results'][0]['Constructor']['name'])


    points_cumsum = np.cumsum(points)
    
    data[driver] = {'index':raceindex,'race':races,'points':points,'cumulative_points':points_cumsum, 'team':team}
    

### Creating the graph:

In [17]:
from bokeh.io import output_notebook, show
from bokeh.layouts import row
from bokeh.palettes import Viridis256
from bokeh.plotting import figure
from bokeh.models import CheckboxGroup, CustomJS, HoverTool, ColumnDataSource
import bokeh
import math

output_notebook()



p = figure()
props = dict(line_width=4, line_alpha=0.7)
callback_string = '{}.visible = {} in checkbox.active;'
code_string = ''
i = 0
sources = []
labels = []
active = []
glyph_dict = {}
colors = bokeh.palettes.viridis(len(data))


for driver in data:
    labels.append(driver.title())
    x = data[driver]['index']
    y = data[driver]['cumulative_points']
    gp = data[driver]['race']
    team = data[driver]['team']
    source =ColumnDataSource(data = {'x':x,'y':y, 'gp':gp, 'team':team})
    sources.append(source)
    line = p.line('x', 'y', source=sources[i], legend=False, color=colors[i], **props, hover_line_color="black")
    hover = HoverTool(tooltips=[('driver', driver.title()),('points', '@y'),('gp','@gp'),('Team','@team')], renderers=[line])
    p.add_tools(hover)
    line.name = driver
    code_string += callback_string.format(driver, str(i))
    glyph_dict.update({driver:line})
    active.append(i)
    i += 1

checkbox = CheckboxGroup(labels=labels, active=active, width=200)
glyph_dict.update({'checkbox':checkbox})
checkbox.callback = CustomJS.from_coffeescript(args=glyph_dict, code=code_string)


p.legend.location = "top_left"
labels_override = dict(zip(data['bottas']['index'], data['bottas']['race']))
p.xaxis.major_label_overrides = labels_override
p.xaxis.major_label_orientation = math.pi/4


layout = row(checkbox, p)
show(layout)

Loading BokehJS ...